# Use BoomSpikeSlab
## First check the result of varbvs on whole genome

In [1]:
res = readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.binomial.whole.genome.prior.varbvs.nologodds.updateb0.rds")

In [2]:
mu0 = sum(res$b0 * res$w)

In [3]:
mu0

[1] 0.7770721

In [4]:
sd0 = sqrt(sum(res$sa * res$w))

In [5]:
sd0

[1] 0.8436502

In [6]:
pi0 = sum((10^(res$logodds) / (1 + 10^(res$logodds))) * res$w)

In [7]:
pi0

[1] 0.0437755

In [8]:
# library(varbvs)
# help(varbvs)

In [9]:
# varbvs::varbvs

In [10]:
library(data.table)
library(reticulate)

In [11]:
library(susieR)
library(BoomSpikeSlab)

Loading required package: Boom
Loading required package: MASS

Attaching package: ‘Boom’

The following object is masked from ‘package:stats’:

    rWishart


Attaching package: ‘BoomSpikeSlab’

The following object is masked from ‘package:stats’:

    knots



In [12]:
X = as.matrix(data.table::fread("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/block_79_137/deletion.genes.block30.for_simu.sample.genes.block_79_137.gz", header = F))

In [13]:
dim(X)

[1] 13412    59

In [14]:
# X = cbind(1, X)

In [15]:
y <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.y"))

In [16]:
storage.mode(X) = 'double'
storage.mode(y) = 'double'

In [39]:
log10(0.05/0.95)

[1] -1.278754

In [29]:
prior <- LogitZellnerPrior(X, y, prior.success.probability = 0.05, expected.model.size = 3, prior.information.weight = 0.05, 
                           diagonal.shrinkage = 0, optional.coefficient.estimate = NULL, prior.inclusion.probabilities = NULL)

In [52]:
names(prior)

[1] "prior.inclusion.probabilities" "mu"                           
[3] "sigma.guess"                   "prior.df"                     
[5] "sigma.upper.limit"             "max.flips"                    
[7] "siginv"                        "prior.success.probability"

In [50]:
prior_glm <- SpikeSlabGlmPrior(X, weight = rep(1/nrow(X), nrow(X)), prior.inclusion.probabilities = rep(1/ncol(X), ncol(X)), expected.model.size = 1, 
                               optional.coefficient.estimate = rep(0, ncol(X)), mean.on.natural.scale = -1, max.flips = 5, diagonal.shrinkage = 0, 
                               prior.information.weight = 0.05)

In [51]:
names(prior_glm)

[1] "prior.inclusion.probabilities" "mu"                           
[3] "sigma.guess"                   "prior.df"                     
[5] "sigma.upper.limit"             "max.flips"                    
[7] "siginv"

In [59]:
length(prior_glm$mu)

[1] 59

In [60]:
ncol(X)

[1] 59

In [18]:
prior1 <- prior

In [19]:
tmp_mu <- c(mu0, rep(0,18), mu0, rep(0,3), rep(0,10), rep(0,7), mu0, rep(0,18))

In [20]:
prior1$prior.inclusion.probabilities <- rep(pi0, ncol(X))
prior1$mu <- rep(mu0, ncol(X))
prior1$sigma.guess <- sd0

In [21]:
length(prior1$mu) == ncol(X)

[1] TRUE

In [58]:
model <- logit.spike(y ~ X, niter = 1000, prior = prior_glm)

ERROR: Error in logit.spike(y ~ X, niter = 1000, prior = prior_glm): length(prior$mu) == ncol(design) is not TRUE


In [32]:
names(model)

[1] "beta"                 "prior"                "xlevels"             
 [4] "call"                 "terms"                "log.likelihood"      
 [7] "deviance.residuals"   "null.log.likelihood"  "fitted.probabilities"
[10] "fitted.logits"        "response"             "training.data"

In [34]:
colSums(model$beta)

XV1        XV2        XV3        XV4        XV5        XV6        XV7 
-1.3975756  0.0000000  0.0000000  0.0000000  0.0000000 -0.4038266  0.0000000 
       XV8        XV9       XV10       XV11       XV12       XV13       XV14 
 0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000 
      XV15       XV16       XV17       XV18       XV19       XV20       XV21 
 0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000 
      XV22       XV23       XV24       XV25       XV26       XV27       XV28 
 0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000 
      XV29       XV30       XV31       XV32       XV33       XV34       XV35 
 0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000 
      XV36       XV37       XV38       XV39       XV40       XV41       XV42 
 0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000 
      XV43       XV44       XV45       XV46       XV47       XV48       XV49 
 0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000 
      XV50       XV51       XV52       XV53       XV54       XV55       XV56 
 0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000 
      XV57       XV58       XV59 
 0.0000000  0.0000000  0.0000000

In [97]:
dim(model$beta)

[1] 100  59

In [98]:
mean(model$beta[, 20])

[1] 0.009948308

In [13]:
mean(model$beta[, 50])

[1] 0.001787244

In [45]:
res1 = susie(X, y, L = 3, scaled_prior_variance = 0.005, estimate_prior_method = "optim")

In [47]:
max(res1$pip)

[1] 0

In [29]:
res2 = readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/block_79_137/deletion.genes.block30.for_simu.sample.genes.block_79_137.varbvs.rds")

In [30]:
names(res2)

[1] "fit"  "mu"   "beta"

In [48]:
max(res2$fit$pip)

[1] 0.02698148

**Make input prior as the format of output of SpikeSlabPrior**

pi: vector of prior.inclusion.probabilities

b: optional.coefficient.estimate

    spike.slab.prior Create a spike and slab prior for use with lm.spike.
    SpikeSlabPrior(x,
    y = NULL,
    expected.r2 = .5,
    prior.df = .01,
    expected.model.size = 1,
    prior.information.weight = .01,
    diagonal.shrinkage = .5,
    optional.coefficient.estimate = NULL,
    max.flips = -1,
    mean.y = mean(y, na.rm = TRUE),
    sdy = sd(as.numeric(y), na.rm = TRUE),
    prior.inclusion.probabilities = NULL,
    sigma.upper.limit = Inf)
    
    This object is intended for use with logit.spike.
    LogitZellnerPrior(
    predictors,
    successes = NULL,
    trials = NULL,
    prior.success.probability = NULL,
    expected.model.size = 1,
    prior.information.weight = .01,
    diagonal.shrinkage = .5,
    optional.coefficient.estimate = NULL,
    max.flips = -1,
    prior.inclusion.probabilities = NULL)

    logit.spike    Spike and slab logistic regression
    logit.spike(formula,
    niter,
    data,
    subset,
    prior = NULL,
    na.action = options("na.action"),
    contrasts = NULL,
    drop.unused.levels = TRUE,
    initial.value = NULL,
    ping = niter / 10,
    nthreads = 0,
    clt.threshold = 2,
    mh.chunk.size = 10,
    proposal.df = 3,
    sampler.weights = c("DA" = .333, "RWM" = .333, "TIM" = .333),
    seed = NULL,
    ...)
    
    lm.spike       Spike and slab regression
    lm.spike(formula,
    niter,
    data,
    subset,
    prior = NULL,
    error.distribution = c("gaussian", "student"),
    contrasts = NULL,
    drop.unused.levels = TRUE,
    model.options = SsvsOptions(),
    ping = niter / 10,
    seed = NULL,
    ...)